In [ ]:
#!pip install checklist

In [2]:
!pip install simpletransformers

     |████████████████████████████████| 249 kB 5.3 MB/s 
     |████████████████████████████████| 342 kB 42.3 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 10.1 MB 31.8 MB/s 
     |████████████████████████████████| 1.2 MB 39.9 MB/s 
     |████████████████████████████████| 1.8 MB 22.8 MB/s 
     |████████████████████████████████| 145 kB 23.2 MB/s 
     |████████████████████████████████| 181 kB 25.4 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 136 kB 20.4 MB/s 
     |████████████████████████████████| 1.1 MB 36.6 MB/s 
     |████████████████████████████████| 212 kB 34.7 MB/s 
     |████████████████████████████████| 127 kB 34.7 MB/s 
     |████████████████████████████████| 144 kB 35.7 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████| 271 kB 41.6 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████

In [86]:
import numpy as np
import checklist
import pandas as pd
import spacy
from checklist.editor import Editor
from checklist.perturb import Perturb
from simpletransformers.classification import ClassificationModel,ClassificationArgs
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,average_precision_score
import warnings
warnings.filterwarnings("ignore")
from checklist.test_types import MFT, INV, DIR

In [4]:
data = pd.read_csv('/content/data/olid-subset-diagnostic-tests.csv')
train_data = pd.read_csv('/content/data/olid-train.csv')

data = data[['text','labels']]
train_data = train_data[['text','labels']]
data.head(3)

,text,labels
0,@USER @USER Who the hell does he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1


In [5]:
nlp = spacy.load('en_core_web_sm')

In [6]:
perturbed_data = list(nlp.pipe(data['text']))

In [7]:
perturbed_data[:3]

[@USER @USER Who the hell does he think he is?,
 #BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL,
 #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!]

In [8]:
np.random.seed(42)
ret_data_typo2 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 2)
ret_data_typo4 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 4)
ret_data_typo6 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 6)
ret_data_typo8 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 8)
ret_data_typo10 = Perturb.perturb(data['text'], Perturb.add_typos, typos = 10)

ret_data_typo2.data[:3][1]

['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
 '#BREAKING. #Greece: Molotov cocktails fly after protest honouring kille dantifa arti... URL via @USER URL']

In [9]:
def make_data_list(data):
  typo_data = []
  for i in range(len(data)):
    typo_data.append(data[i][1]) # tweet with typos added
  
  return typo_data

In [10]:
typo_data2 = make_data_list(ret_data_typo2.data)
typo_data4 = make_data_list(ret_data_typo4.data)
typo_data6 = make_data_list(ret_data_typo6.data)
typo_data8 = make_data_list(ret_data_typo8.data)
typo_data10 = make_data_list(ret_data_typo10.data)

In [11]:
print(typo_data2[0])
print(typo_data10[0])

@USER @USER Who the hell doe she thinkh e is?
U@SER U@SE RhWo th eehl dloes he think h eis?


In [12]:
data_typo2 = pd.DataFrame({'text':typo_data2,'label':data['labels']})
data_typo4 = pd.DataFrame({'text':typo_data4,'label':data['labels']})
data_typo6 = pd.DataFrame({'text':typo_data6,'label':data['labels']})
data_typo8 = pd.DataFrame({'text':typo_data8,'label':data['labels']})
data_typo10 = pd.DataFrame({'text':typo_data10,'label':data['labels']})

data_typo2.head()

,text,label
0,@USER @USER Who the hell doe she thinkh e is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,"#OrrinHatch I can’t eblieve this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 lost my acca o nth efirst fucking fight cba,1


In [13]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.overwrite_output_dir= True
model_args.output_dir = '/content/drive/MyDrive/NLP/error_analyse_model'


model = ClassificationModel("bert", 'bert-base-cased', args=model_args)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [14]:
model.train_model(train_data)

  0%|          | 0/13240 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1655 [00:00<?, ?it/s]

(1655, 0.49156262492845426)

In [15]:
predictions2, raw_outputs2 = model.predict(list(data_typo2['text']))
predictions4, raw_outputs4 = model.predict(list(data_typo4['text']))
predictions6, raw_outputs6 = model.predict(list(data_typo6['text']))
predictions8, raw_outputs8 = model.predict(list(data_typo8['text']))
predictions10, raw_outputs10 = model.predict(list(data_typo10['text']))

pred, out = model.predict(list(data['text']))

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [16]:
predicted_typo_labels2 = pd.DataFrame({'text':data_typo2.text, 'labels': predictions2})
predicted_typo_labels2.head()

,text,labels
0,@USER @USER Who the hell doe she thinkh e is?,1
1,#BREAKING. #Greece: Molotov cocktails fly afte...,0
2,"#OrrinHatch I can’t eblieve this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,0
4,0-1 lost my acca o nth efirst fucking fight cba,1


In [17]:
predicted_typo_labels10 = pd.DataFrame({'text':data_typo10.text, 'labels': predictions10})
predicted_typo_labels10.head()

,text,labels
0,U@SER U@SE RhWo th eehl dloes he think h eis?,0
1,#BREAKING. #Greece :oMlotov cocktails fly afte...,0
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USERI 'll uset hat one the next tim eim...,0
4,0-1l ost m ycaac o nhet first fucikgn fight cba,0


In [18]:
result2, model_outputs2, wrong_predictions2 = model.eval_model(data_typo2, report_typo2 = classification_report)
result4, model_outputs4, wrong_predictions4 = model.eval_model(data_typo4, report_typo4 = classification_report)
result6, model_outputs6, wrong_predictions6 = model.eval_model(data_typo6, report_typo6 = classification_report)
result8, model_outputs8, wrong_predictions8 = model.eval_model(data_typo8, report_typo8 = classification_report)
result10, model_outputs10, wrong_predictions10 = model.eval_model(data_typo10, report_typo10 = classification_report)

result_d, model_outputs_d, wrong_predictions_d = model.eval_model(data, report_normaldata = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [19]:
print("Original data prediction")
print(result_d['report_normaldata'])
print("\n")
print("With 2 typos")
print(result2['report_typo2'])
print("\n")
print("With 4 typos")
print(result4['report_typo4'])
print("\n")
print("With 6 typos")
print(result6['report_typo6'])
print("\n")
print("With 8 typos")
print(result8['report_typo8'])
print("\n")
print("With 10 typos")
print(result10['report_typo10'])


Original data prediction
              precision    recall  f1-score   support

         0.0       0.71      0.98      0.82        50
         1.0       0.97      0.60      0.74        50

    accuracy                           0.79       100
   macro avg       0.84      0.79      0.78       100
weighted avg       0.84      0.79      0.78       100



With 2 typos
              precision    recall  f1-score   support

         0.0       0.70      0.94      0.80        50
         1.0       0.91      0.60      0.72        50

    accuracy                           0.77       100
   macro avg       0.81      0.77      0.76       100
weighted avg       0.81      0.77      0.76       100



With 4 typos
              precision    recall  f1-score   support

         0.0       0.68      0.94      0.79        50
         1.0       0.90      0.56      0.69        50

    accuracy                           0.75       100
   macro avg       0.79      0.75      0.74       100
weighted avg       

In [20]:
print(model_outputs10[:5])

[[ 0.41772461 -1.56835938]
 [-0.03344727 -1.06738281]
 [-1.16113281  0.93115234]
 [-0.49169922 -0.62304688]
 [ 0.34423828 -1.48828125]]


In [21]:
print("Confusion Matrix (2 typos): ")
print(confusion_matrix(data['labels'], predictions2))
print("\n")
print("Confusion Matrix (4 typos): ")
print(confusion_matrix(data['labels'], predictions4))
print("\n")
print("Confusion Matrix (6 typos): ")
print(confusion_matrix(data['labels'], predictions6))
print("\n")
print("Confusion Matrix (8 typos): ")
print(confusion_matrix(data['labels'], predictions8))
print("\n")
print("Confusion Matrix (10 typos): ")
print(confusion_matrix(data['labels'], predictions10))
print("\n")

Confusion Matrix (2 typos): 
[[47  3]
 [20 30]]


Confusion Matrix (4 typos): 
[[47  3]
 [22 28]]


Confusion Matrix (6 typos): 
[[48  2]
 [24 26]]


Confusion Matrix (8 typos): 
[[49  1]
 [23 27]]


Confusion Matrix (10 typos): 
[[46  4]
 [30 20]]




### Provide 3 examples when the model failed to assign the correct label after perturbation.

In [22]:
print(data['labels'][:10])

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: labels, dtype: int64


In [23]:
print(predicted_typo_labels10['labels'][:10])

0    0
1    0
2    1
3    0
4    0
5    1
6    0
7    0
8    0
9    1
Name: labels, dtype: int64


In [24]:
failed_texts=[]
failed_labels=[]
for i in range(len(data)):
  if predicted_typo_labels10['labels'][i] != data['labels'][i]:
    failed_texts.append(predicted_typo_labels10['text'][i])
    failed_labels.append(predicted_typo_labels10['labels'][i])

In [25]:
failed_examples = pd.DataFrame({'text':failed_texts, 'labels': failed_labels})
failed_examples

,text,labels
0,U@SER U@SE RhWo th eehl dloes he think h eis?,0
1,#BREAKING. #Greece :oMlotov cocktails fly afte...,0
2,@USER @USERI 'll uset hat one the next tim eim...,0
3,0-1l ost m ycaac o nhet first fucikgn fight cba,0
4,#hCristian #America – If we go by #Trump’s exa...,0
5,@USER U@SER @USER She is the most disignenuous...,0
6,#Democrats #Liberals you are being t#hreaetned...,0
7,#LiberalHpyocrsiy #criminal #democrat #evidenc...,0
8,(crt1ikal voice) emsf m ayss chekes,0
9,t*ge sall teh ibthce*s,0


# 6. Negation *(4.5 points)* 

In [26]:
perturbed_data = list(nlp.pipe(data['text']))

In [28]:
ret = Perturb.perturb(perturbed_data, Perturb.add_negation) # negated data
ret.data[:5]

[['@USER @USER Who the hell does he think he is?',
  "@USER @USER Who the hell doesn't he think he is?"],
 ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
  "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL"],
 ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
  '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'],
 ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes.

In [29]:
negated_text = []
for i in range(len(ret.data)):
  negated_text.append(ret.data[i][1])

negated_text[:5]

["@USER @USER Who the hell doesn't he think he is?",
 "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL",
 '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
 "@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can't choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.",
 "0-1 didn't lose my acca on the first fucking fight cba"]

In [30]:
neg_predictions, neg_raw_outputs = model.predict(negated_text)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [31]:
predicted_neg_labels = pd.DataFrame({'text':negated_text, 'labels': data['labels']})
predicted_neg_labels.head()

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails don't fl...,1
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 didn't lose my acca on the first fucking f...,1


In [32]:
result, model_outputs, wrong_predictions = model.eval_model(predicted_neg_labels, neg_report = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [33]:
print(result['neg_report'])

              precision    recall  f1-score   support

         0.0       0.72      0.96      0.82        50
         1.0       0.94      0.62      0.75        50

    accuracy                           0.79       100
   macro avg       0.83      0.79      0.78       100
weighted avg       0.83      0.79      0.78       100



In [34]:
print("Confusion Matrix : ")
print(confusion_matrix(data['labels'], neg_predictions))

Confusion Matrix : 
[[48  2]
 [19 31]]


### Check the first 10 negated messages. For which of these negated messages should the label be flipped, in your opinion?

In [35]:
predicted_neg_labels['text'][:10]

0     @USER @USER Who the hell doesn't he think he is?
1    #BREAKING. #Greece: Molotov cocktails don't fl...
2    #OrrinHatch I can’t believe this sexist , clue...
3    @USER @USER I'll use that one the next time im...
4    0-1 didn't lose my acca on the first fucking f...
5    #Bakersfield is not why we need gun control! S...
6    #Christian #America – If we go by #Trump’s exa...
7    @USER @USER @USER She is not the most disingen...
8    #Democrats #Liberals you are being #threatened...
9    699. Just didn't want to tell you you should H...
Name: text, dtype: object

In [36]:
predicted_neg_labels_df = pd.DataFrame({'text':negated_text, 'labels': neg_predictions})
predicted_neg_labels_df.head()

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,#BREAKING. #Greece: Molotov cocktails don't fl...,0
2,"#OrrinHatch I can’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,0
4,0-1 didn't lose my acca on the first fucking f...,1


In [37]:
failed_neg_texts=[]
failed_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels_df['labels'][i] != data['labels'][i]:
    failed_neg_texts.append(predicted_neg_labels['text'][i])
    failed_neg_labels.append(predicted_neg_labels['labels'][i])

In [38]:
failed_neg_examples = pd.DataFrame({'text':failed_neg_texts, 'labels': failed_neg_labels})
failed_neg_examples

,text,labels
0,#BREAKING. #Greece: Molotov cocktails don't fl...,1
1,@USER @USER I'll use that one the next time im...,1
2,#Christian #America – If we go by #Trump’s exa...,1
3,@USER @USER @USER She is not the most disingen...,1
4,#Democrats #Liberals you are being #threatened...,1
5,#Antifa: don't take note of how to protest wit...,1
6,#ANTIFA are not bedfellows with The Democratic...,1
7,@USER Antifa doesn't have TS level influence. ...,1
8,@USER @USER @USER @USER doesn't beat the hell ...,1
9,"#ConfirmKavanugh now, stall tactics are not DC...",1


In [39]:
success_neg_texts=[]
success_neg_labels=[]
for i in range(len(data)):
  if predicted_neg_labels_df['labels'][i] == data['labels'][i]:
    success_neg_texts.append(predicted_neg_labels['text'][i])
    success_neg_labels.append(predicted_neg_labels['labels'][i])

In [40]:
successful_neg_examples = pd.DataFrame({'text':success_neg_texts, 'labels': success_neg_labels})
successful_neg_examples

,text,labels
0,@USER @USER Who the hell doesn't he think he is?,1
1,"#OrrinHatch I can’t believe this sexist , clue...",1
2,0-1 didn't lose my acca on the first fucking f...,1
3,#Bakersfield is not why we need gun control! S...,1
4,699. Just didn't want to tell you you should H...,1
...,...,...
74,@USER He is obviously getting not suspended. H...,0
75,#Canada - EXCLUSIVE: #Trudeau #Liberals don't ...,0
76,@USER @USER ...than why didn't you show us how...,0
77,@USER @USER @USER You don't have yet to answer...,0


## 7. Creating negated examples

In [41]:
editor = Editor()

In [42]:
hate_mask = editor.template("I hate {mask}")
dont_hate_mask = editor.template("I don't hate {mask}")

hate_nationality = editor.template("I hate {nationality}")
dont_hate_nationality = editor.template("I don't hate {nationality}")

hate_religion = editor.template("I hate {religion}")
dont_hate_religion = editor.template("I don't hate {religion}")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

In [43]:
hate_mask = hate_mask.data
dont_hate_mask = dont_hate_mask.data

hate_nationality = hate_nationality.data
dont_hate_nationality = dont_hate_nationality.data

hate_religion = hate_religion.data
dont_hate_religion = dont_hate_religion.data

In [51]:
dont_hate_mask[10:15]

["I don't hate anybody",
 "I don't hate Muslims",
 "I don't hate Jews",
 "I don't hate gays",
 "I don't hate religion"]

In [52]:
hate_religion[:5]

['I hate Christianity',
 'I hate Judaism',
 'I hate Islam',
 'I hate Buddhism',
 'I hate Zoroastrianism']

In [53]:
hatemask_predictions, hatemask_outputs = model.predict(hate_mask)
donthatemask_predictions, donthatemask_outputs = model.predict(dont_hate_mask)

hatenati_predictions, hatenati_outputs = model.predict(hate_nationality)
donthatenati_predictions, donthatenati_outputs = model.predict(dont_hate_nationality)

hatereligion_predictions, hatereligion_outputs = model.predict(hate_religion)
donthatereligion_predictions, donthatereligion_outputs = model.predict(dont_hate_religion)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/215 [00:00<?, ?it/s]

...

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
hatemask_df = pd.DataFrame({'text':hate_mask,'label':hatemask_predictions})
hatemask_df.head()

,text,label
0,I hate it,0
1,I hate capitalism,1
2,I hate Trump,0
3,I hate racism,1
4,I hate them,1


In [57]:
donthatemask_df = pd.DataFrame({'text':dont_hate_mask,'label':donthatemask_predictions})
donthatemask_df.head()

,text,label
0,I don't hate them,1
1,I don't hate you,1
2,I don't hate him,0
3,I don't hate people,1
4,I don't hate her,0


In [58]:
hatenati_df = pd.DataFrame({'text':hate_nationality,'label':hatenati_predictions})
hatenati_df.head()

,text,label
0,I hate Chinese,1
1,I hate Indian,1
2,I hate American,1
3,I hate Indonesian,1
4,I hate Pakistani,1


In [59]:
donthatenati_df = pd.DataFrame({'text':dont_hate_nationality,'label':donthatenati_predictions})
donthatenati_df.head()

,text,label
0,I don't hate Chinese,1
1,I don't hate Indian,1
2,I don't hate American,1
3,I don't hate Indonesian,0
4,I don't hate Pakistani,1


In [60]:
hatereligion_df = pd.DataFrame({'text':hate_religion,'label':hatereligion_predictions})
hatereligion_df.head()

,text,label
0,I hate Christianity,1
1,I hate Judaism,0
2,I hate Islam,1
3,I hate Buddhism,1
4,I hate Zoroastrianism,0


In [61]:
donthatereligion_df = pd.DataFrame({'text':dont_hate_religion,'label':donthatereligion_predictions})
donthatereligion_df.head()

,text,label
0,I don't hate Christianity,0
1,I don't hate Judaism,0
2,I don't hate Islam,0
3,I don't hate Buddhism,0
4,I don't hate Zoroastrianism,0


In [62]:
hatereligion_df[hatereligion_df['label'] == 0] # wrong

,text,label
1,I hate Judaism,0
4,I hate Zoroastrianism,0
6,I hate Sikhism,0
7,I hate Shintoism,0
9,I hate Taoism,0
10,I hate Confucianism,0
11,I hate Jain,0
12,I hate Atheism,0
13,I hate Agnosticism,0


In [64]:
hatenati_df[hatenati_df['label'] == 0].head(10)

,text,label
25,I hate Kenyan,0
36,I hate Canadian,0
37,I hate Moroccan,0
50,I hate Cameroonian,0
53,I hate Sri Lankan,0
54,I hate Burkinabé,0
56,I hate Malian,0
63,I hate Ecuadorian,0
71,I hate Rwandan,0
80,I hate Greek,0


In [65]:
hatemask_df[hatemask_df['label'] == 0].head(10)

,text,label
0,I hate it,0
2,I hate Trump,0
6,I hate politics,0
8,I hate that,0
14,I hate […],0
20,I hate war,0
22,I hate religion,0
26,I hate feminism,0
27,I hate .,0
28,I hate Facebook,0


### 3 examples when the model fails to assign the correct label *(choose both from masking and lexicon suggestions)*

In [66]:
print(list(editor.lexicons.keys()))

['male', 'female', 'first_name', 'first_pronoun', 'last_name', 'country', 'nationality', 'city', 'religion', 'religion_adj', 'sexual_adj', 'sentiment', 'country_city', 'male_from', 'female_from', 'last_from']


In [72]:
list(editor.lexicons.keys())[6]

'nationality'

In [67]:
editor.suggest('I hate {mask}.')[:5]

['it', 'them', 'capitalism', 'cats', 'politics']

In [70]:
editor.related_words('I hate Canadian.', 'Canadian')[:5]

['people', 'war', 'change', 'feminism', 'life']

In [71]:
editor.related_words('I hate Islam.', 'Islam')[:5]

['politics', 'people', 'death', 'change', 'feminism']

In [82]:
', '.join(editor.suggest('This is not {a:mask} {thing}.', thing=list(editor.lexicons.keys())[6]))

'French, European, American, Arab, Italian, African, English, alien, Iranian, Irish, official, Indian, Asian, EU, Islamic, Israeli, Australian, honorary, Austrian, Arabic, individual, ordinary, Icelandic, acceptable, invented, independent, indigenous, anonymous, Indonesian, ethnic, advanced, automatic, enemy, Egyptian, illegal, exotic, imperial, old, ideal, international, exceptional, exclusive, endangered, Afghan, foreign, Armenian, authentic, new, Eastern, alternative, assumed, easy, oppressed, Japanese, British, open, immigrant, German, apartheid, Ethiopian, identifiable, single, island, established, adopted, original, important, aggressive, Western, existing, dual, attractive, accepted, inferior, Argentine, Hungarian, artificial, alternate, offensive, common, equal, isolated, actual, unusual, imaginary, special, Russian, Chinese, active, particular, allied, Iraqi, Ottoman, undocumented, ancestral, ancient, Spanish, Oriental, honorable, Greek, Turkish, legitimate, arbitrary, unknown

In [110]:
pos = ['like','love', 'do not hate', 'respect', 'admire', 'adore', 'relish']
neg = ['hate','dislike', 'do not like','deset','distaste','hatred','despise']

In [112]:
ret = editor.template('I {a:pos} {nationality}.', pos=pos, labels=0, save=True, nsamples=100)
ret += editor.template('I {a:neg} {nationality}.', neg=neg, labels=1, save=True, nsamples=100)


In [113]:
test = MFT(ret.data, labels=ret.labels, name='Simple negation',
           capability='Negation', description='Very simple negations.')

In [114]:
from pattern.en import sentiment

In [115]:
import numpy as np
def predict_proba(inputs):
    p1 = np.array([(sentiment(x)[0] + 1)/2. for x in inputs]).reshape(-1, 1)
    p0 = 1- p1
    return np.hstack((p0, p1))

In [118]:
predict_proba(['hate', 'do not hate'])


array([[0.9, 0.1],
       [0.3, 0.7]])

In [119]:
from checklist.pred_wrapper import PredictorWrapper
wrapped_pp = PredictorWrapper.wrap_softmax(predict_proba)

In [120]:
wrapped_pp(['hate'])

(array([0]), array([[0.9, 0.1]]))

In [121]:
test.run(wrapped_pp)

Predicting 200 examples


In [122]:
test.summary()

Test cases:      200
Fails (rate):    130 (65.0%)

Example fails:
0.1 I a hate Syrian.
----
0.3 I a hate Greek.
----
0.5 I a despise Indonesian.
----
